In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import numexpr as ne
%matplotlib qt5

In [2]:
import sys
import time

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [5]:
from backprojection.tools import sb1, wa, pulse, pulse2

In [6]:
fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
c = 3e8
kc = 4 * np.pi * fc / c
fs = 10e6

rail = 0
truck = 1
plane = 2
vehicule = plane

if vehicule == rail:
    T = 5e-3
    rampsPerFile = 200
    V = 0.1
if vehicule == truck:
    T = 5e-3
    rampsPerFile = 200
    V = 3
if vehicule == plane:
    T = 600e-6
    rampsPerFile = 1500
    V = 40

alpha = 2 * (fMAX - fmin) / T
samplesPerRamp = int( np.ceil( T * fs ) )
samplesPerFile = rampsPerFile * samplesPerRamp
samplesPerUpRamp = int( samplesPerRamp / 2 )

print("T = {}, samplesPerRamp = {:.2f}, samplesPerFile = {:.2f}".format(T, samplesPerRamp, samplesPerFile))

t1 = np.arange( 0, T, 1/fs)
t2 = np.arange( -T/2, T/2, 1/fs)

T = 0.0006, samplesPerRamp = 6000.00, samplesPerFile = 9000000.00


# Build data

In [7]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook

In [8]:
nbFiles = 20

In [9]:
hPlane = 90
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = V * T * nbFiles * rampsPerFile / 2
noise = 1
print("rg_corner = {:.2f}, gr_corner = {:.2f}, az_corner = {:.2f}".format(rg_corner, gr_corner, az_corner))

rg_corner = 100.00, gr_corner = 100.00, az_corner = 360.00


In [10]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 720, phi = 74.5°


In [11]:
adc = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp), dtype=complex )
for ramp in range( nbFiles * rampsPerFile):
    azn = ramp * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    adc[ ramp,  0 : samplesPerRamp ] = sb1( t1, Rn, alpha, fc ) * wa( azn - az_corner, rg_corner ) \
    + np.random.randn( samplesPerRamp ) * noise / (Rn**2)
    file = ramp/rampsPerFile
    if file == np.floor(file):
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, azn = {:.2f}, wa = {:.1f}, phi = {:.1f}, Rn = {:.1f}".format( 
            file, azn, wa( azn - az_corner, rg_corner ), phi, Rn ) )

0.0, azn = 0.00, wa = 0.0, phi = 74.5, Rn = 373.6
1.0, azn = 36.00, wa = 0.0, phi = 72.8, Rn = 339.1
2.0, azn = 72.00, wa = 0.0, phi = 70.9, Rn = 304.9
3.0, azn = 108.00, wa = 0.0, phi = 68.4, Rn = 271.1
4.0, azn = 144.00, wa = 0.0, phi = 65.2, Rn = 238.0
5.0, azn = 180.00, wa = 0.0, phi = 60.9, Rn = 205.9
6.0, azn = 216.00, wa = 0.0, phi = 55.2, Rn = 175.3
7.0, azn = 252.00, wa = 0.0, phi = 47.2, Rn = 147.2
8.0, azn = 288.00, wa = 0.2, phi = 35.8, Rn = 123.2
9.0, azn = 324.00, wa = 0.7, phi = 19.8, Rn = 106.3
10.0, azn = 360.00, wa = 1.0, phi = 0.0, Rn = 100.0
11.0, azn = 396.00, wa = 0.7, phi = 19.8, Rn = 106.3
12.0, azn = 432.00, wa = 0.2, phi = 35.8, Rn = 123.2
13.0, azn = 468.00, wa = 0.0, phi = 47.2, Rn = 147.2
14.0, azn = 504.00, wa = 0.0, phi = 55.2, Rn = 175.3
15.0, azn = 540.00, wa = 0.0, phi = 60.9, Rn = 205.9
16.0, azn = 576.00, wa = 0.0, phi = 65.2, Rn = 238.0
17.0, azn = 612.00, wa = 0.0, phi = 68.4, Rn = 271.1
18.0, azn = 648.00, wa = 0.0, phi = 70.9, Rn = 304.9
19.0, az

# Backprojection

In [12]:
analyticSignal = adc[:, 0:samplesPerUpRamp:2]
srf = np.fft.ifft( analyticSignal, axis=1 )
print( "srf.shape = {}".format(srf.shape))

srf.shape = (30000, 1500)


## Image geometry

In [13]:
x_min = 200
x_max = 500
r_min = 80
r_max = 120

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 1
d_r = 1

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size, 1 )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( 1, r.size )

## Range focalization

In [14]:
overSamplingRatio = 10
Nf = srf.shape[1]
Nover = overSamplingRatio * Nf
rangeResolution = c / (2 * B)
r_base = np.arange( 0, Nf ) * rangeResolution
r_over = np.arange( 0, Nover ) * rangeResolution / overSamplingRatio
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

range from 0.00m to 1499.90m, resolution = 1.0m, oversampled = 0.1m, 


In [15]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V

print( "x.shape = {}, r.shape = {}".format( x.shape, r.shape ) )
print( "x from {} to {}, r from {} to {}".format(x[0,0], x[-1,0], r[0,0], r[0,-1]) )

phi = 6 * np.pi / 180

nbLoops = xa_vec.shape[0]
progress = int( nbLoops / 10 )

x.shape = (300, 1), r.shape = (1, 40)
x from 200 to 499, r from 80 to 119


In [16]:
t = time.time()

img  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img.shape ) )

loop = 0
endK = 10000
for k in range( endK ):
    xa =xa_vec[k]
    loop = k
#     if xa in xa_vec[::progress]:
#         print( "{:.2f} / {:.2f}".format(xa, xa_vec[-1] ) )

    if k == endK-1:
        d = (r[0,20]**2 + (xa-x[160,0])**2 )**0.5
        aux1 = np.exp( 1j * kc * d )
        y = signal.resample( srf[loop,:], Nover  )
        aux2 = np.interp( d, r_over, signal.resample( srf[loop,:], Nover  ) )
        aux3 = pulse2( (xa-x[160,0]) / (r[0,20]*np.tan(phi)) )
        aux4 = aux1 * aux2 * aux3
        print("k = {}   d = {}   aux1 = {}   aux2 = {}\n   aux3 = {}   aux4 = {}".format( k, d, aux1, aux2, aux3, aux4) )
        print("srf = {:.10f}   srf = {:.10f}\ny = {:.10f}   y = {:.10f}\n".format(srf[k, 0], srf[k, 1], y[1000], y[1001]))

    img += np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
    * np.interp( (r**2 + (xa-x)**2 )**0.5, r_over, signal.resample( srf[loop,:], Nover  ) ) \
    * pulse( (xa-x) / (r*np.tan(phi)) )
    
    loop += 1
# np.exp( -1j * kc * r)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

img.shape = (300, 40)
k = 9999   d = 156.22343158438173   aux1 = (-0.6405433195086333-0.7679220376007325j)   aux2 = (-0.014344949581667005-0.001822033857322327j)
   aux3 = 0   aux4 = 0j
srf = -0.0000227856-0.0000111447j   srf = -0.0000227349-0.0000116245j
y = -0.0000670671-0.0000163184j   y = -0.0000348642-0.0000082721j

execution time = 28.74583077430725


In [ ]:
img[0,0]

In [ ]:
d.dtype

In [ ]:
srf[14995,0], srf[14995,1]

In [ ]:
analyticSignal[0:5, 0]

In [ ]:
r[0,20]*np.tan(phi)

In [ ]:
xa_vec.size

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

In [17]:
data_dir = "/home/pleroy/DATA/SIMU"
np.save( data_dir + "/analyticSignal_{}_{}".format(0, nbFiles-1), analyticSignal )

In [ ]:
analyticSignal[0, 0:5]

## Try to optimize

In [ ]:
%load_ext line_profiler

In [ ]:
def interp( x, xp, fp, dx, idx1, idx2, y ):
    np.floor( x / dx, out=idx1, casting='unsafe' )
    np.ceil( x / dx, out=idx2, casting='unsafe' )
    y2 = fp[idx2]
    y1 = fp[idx1]
    x1 = xp[idx1]
    ne.evaluate("(y2-y1) / dx * (x - x1) + y1" , out=y )

In [ ]:
xa = 0
y = np.zeros( img.shape, dtype=complex )
idx1 = np.zeros( img.shape, dtype=int )
idx2 = np.zeros( img.shape, dtype=int )
dx = r_over[1] - r_over[0]
print( "dx = {}".format(dx) )
d = ne.evaluate( "(r**2 + (xa-x)**2 )**0.5" )
srf_over = signal.resample( srf[0,:], nbPointsResampled  )
interp( d, r_over, srf_over, dx, idx1, idx2, y )

In [ ]:
plt.figure()
plt.plot(r_base, np.real(srf[0,:]), 'oy')
plt.plot(r_over, np.real(srf_over))
plt.plot(d[0,:], np.real(y[0,:]), '.-')

In [ ]:
y = np.zeros( img.shape, dtype=complex )
idx1 = np.zeros( img.shape, dtype=int )
idx2 = np.zeros( img.shape, dtype=int )
img  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img.shape ) )
dx = r_over[1] - r_over[0]
print( "dx = {}".format(dx) )

def backProjection( xa_vec, img ):
    t = time.time()
    loop = 0
    img_xa = np.zeros( img.shape, dtype=complex )
    d = np.zeros( img.shape, dtype=float )
    for xa in xa_vec:
        if xa in xa_vec[::progress]:
            print( "{:.2f} / {:.2f}".format(xa, xa_vec[-1] ) )
        ne.evaluate( "(r**2 + (xa-x)**2 )**0.5", out=d )
        ne.evaluate( "exp( 1j * kc * d )", out=img_xa )
        img_xa *= pulse( (xa-x) / (r*np.tan(phi)) )
        srf_over = signal.resample( srf[loop,:], nbPointsResampled  )
        interp( d, r_over, srf_over, dx, idx1, idx2, y )
        img_xa *= y
        #img_xa *= np.interp( d, r_over, signal.resample( srf[loop,:], nbPointsResampled  ) )
        img += img_xa
        loop += 1
    elapsed = time.time() - t
    print("execution time = " + str(elapsed))

In [ ]:
%lprun -f backProjection backProjection( xa_vec, img )

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

In [ ]:
d = np.zeros( img.shape, dtype=float )

In [ ]:
d.dtype

# Save data

In [ ]:
data_dir = "/home/pleroy/DATA/SIMU"
np.save( data_dir + "/analyticSignal_0_{}".format(nbFiles-1), analyticSignal )

# Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [ ]:
y0 = 0.1
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [ ]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

In [ ]:
a = 0.603354564402 # y0 = 0.5
a = 0.90792862378 # y0 = 0.1
b = a * 180 / (30 * np.pi)

In [ ]:
x = 30 * np.pi / 180
np.sinc( b * x )

# fft2D

In [ ]:
plt.figure()
plt.imshow(20*np.log10(np.abs(np.fft.fftshift(np.fft.fft2(img)))))

In [ ]:
plt.figure()
plt.imshow(20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(img,axis=1), axes=1))))

In [ ]:
plt.figure()
plt.imshow(20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(img,axis=0), axes=0))))

In [ ]:
plt.figure()
plt.plot(20*np.log10(np.abs(np.fft.fftshift(np.fft.ifft(np.real(signal.resample(srf[0,:],nbPointsResampled)))))))